# 2. Introduction to FastAPI
In this notebook, we will start building our own APIs using **FastAPI**.

## Introduction
**FastAPI** is a modern, fast (high-performance), web framework for building APIs with Python 3.6+ based on standard Python type hints.

### Why FastAPI?
- **Fast**: Very high performance, on par with NodeJS and Go (thanks to Starlette and Pydantic).
- **Fast to code**: Increase the speed to develop features by about 200% to 300%.
- **Fewer bugs**: Reduce about 40% of human (developer) induced errors.
- **Intuitive**: Great editor support. Completion everywhere. Less time debugging.
- **Easy**: Designed to be easy to use and learn. Less time reading docs.
- **Short**: Minimize code duplication. Multiple features from each parameter declaration.
- **Robust**: Get production-ready code. With automatic interactive documentation.
- **Standards-based**: Based on (and fully compatible with) the open standards for APIs: OpenAPI (previously known as Swagger) and JSON Schema.

### Useful Resources
- [FastAPI Documentation](https://fastapi.tiangolo.com/)
- [Pydantic Documentation](https://docs.pydantic.dev/latest/)
- [Starlette Documentation](https://www.starlette.io/)

In [1]:
# Imports
from fastapi import FastAPI
from fastapi.testclient import TestClient

## 2.1 Learning Goals
- Create a basic FastAPI application.
- Understand the `@app.get` decorator.
- Use `TestClient` to test the API within the notebook.
- Understand Path Parameters and Query Parameters in FastAPI.
- Learn about **Swagger UI** (Automatic Documentation).

### Notebook roadmap
- Build a tiny API (`/`) to see the FastAPI + `TestClient` loop.
- Explore auto-docs (Swagger UI) conceptually even though we run in-notebook.
- Add path params (`/items/{item_id}`) and see validation errors FastAPI gives you for free.
- Add query params (`/users/`) and optional defaults.
- Finish with a small exercise to practice combining both kinds of parameters.
Tip: because the `app` object is reused, run cells in order so earlier routes are defined before later requests.


## 2.2 Your First API
We will create a simple API with a single endpoint that returns a "Hello World" message.

In [2]:
# 1. Create the FastAPI app instance
app = FastAPI()

# 2. Define a path operation decorator
@app.get("/")
def read_root():
    return {"message": "Hello World"}

# 3. Create a TestClient to interact with the app
client = TestClient(app)

# 4. Make a request to the API
response = client.get("/")
print(response.status_code)
print(response.json())

200
{'message': 'Hello World'}


## 2.3 Automatic Documentation (Swagger UI)
One of the best features of FastAPI is that it automatically generates interactive API documentation for you.

If you were running this code in a terminal (using `uvicorn main:app --reload`), you could visit `http://127.0.0.1:8000/docs` in your browser to see the **Swagger UI**.

Swagger UI allows you to:
- See all available endpoints.
- See what parameters they expect.
- Execute requests directly from the browser.

Since we are in a notebook, we will use `TestClient` to simulate requests, but keep in mind that in a real project, Swagger UI is your best friend for testing and documentation.

## 2.4 Path Parameters
Path parameters are parts of the URL path that are variable.
Example: `/items/5` where `5` is the `item_id`.

In [3]:
@app.get("/items/{item_id}")
def read_item(item_id: int):
    return {"item_id": item_id}

# Test it
response = client.get("/items/42")
print(response.json())

# Test with invalid type (string instead of int)
response = client.get("/items/foo")
print(response.status_code)
print(response.json())

{'item_id': 42}
422
{'detail': [{'type': 'int_parsing', 'loc': ['path', 'item_id'], 'msg': 'Input should be a valid integer, unable to parse string as an integer', 'input': 'foo'}]}


## 2.5 Query Parameters
Function parameters that are not part of the path are automatically interpreted as "query" parameters.

In [4]:
@app.get("/users/")
def read_user(user_id: int, q: str = None):
    return {"user_id": user_id, "q": q}

# Test with query param
response = client.get("/users/?user_id=1&q=somequery")
print(response.json())

# Test without optional query param
response = client.get("/users/?user_id=1")
print(response.json())

{'user_id': 1, 'q': 'somequery'}
{'user_id': 1, 'q': None}


## 2.6 Exercise: Personalized Greeting
**Task**:
1. Create a new endpoint `/greet/{name}`.
2. It should take `name` as a path parameter.
3. It should take an optional query parameter `is_formal` (boolean, default `False`).
4. If `is_formal` is `True`, return `{"message": "Good day, {name}"}`.
5. Otherwise, return `{"message": "Hi, {name}"}`.

In [5]:
# Create the endpoint with path parameter and optional query parameter
@app.get("/greet/{name}")
def greet(name: str, is_formal: bool = False):
    """
    Greet a person by name.
    
    Args:
        name: The person's name (path parameter)
        is_formal: Whether to use formal greeting (query parameter, default False)
    
    Returns:
        A greeting message
    """
    if is_formal:
        return {"message": f"Good day, {name}"}
    else:
        return {"message": f"Hi, {name}"}

# Test the endpoint with various scenarios
print("=== Test 1: Informal greeting ===")
response = client.get("/greet/Bob")
print(f"Status Code: {response.status_code}")
print(f"Response: {response.json()}")

print("\n=== Test 2: Formal greeting (is_formal=true) ===")
response = client.get("/greet/Bob?is_formal=true")
print(f"Status Code: {response.status_code}")
print(f"Response: {response.json()}")

print("\n=== Test 3: Formal greeting (is_formal=false) ===")
response = client.get("/greet/Alice?is_formal=false")
print(f"Status Code: {response.status_code}")
print(f"Response: {response.json()}")

print("\n=== Test 4: Different name with formal greeting ===")
response = client.get("/greet/Dr.Smith?is_formal=true")
print(f"Status Code: {response.status_code}")
print(f"Response: {response.json()}")

=== Test 1: Informal greeting ===
Status Code: 200
Response: {'message': 'Hi, Bob'}

=== Test 2: Formal greeting (is_formal=true) ===
Status Code: 200
Response: {'message': 'Good day, Bob'}

=== Test 3: Formal greeting (is_formal=false) ===
Status Code: 200
Response: {'message': 'Hi, Alice'}

=== Test 4: Different name with formal greeting ===
Status Code: 200
Response: {'message': 'Good day, Dr.Smith'}


Here's what was added:

## Implementation Details

**Endpoint Created:** `/greet/{name}`

**Features:**
1. **Path Parameter**: `name` (string) - captures the person's name from the URL
2. **Query Parameter**: `is_formal` (boolean, default `False`) - optional parameter to control greeting style
3. **Logic**:
   - If `is_formal=True`: Returns `{"message": "Good day, {name}"}`
   - If `is_formal=False` (or not provided): Returns `{"message": "Hi, {name}"}`

**Test Cases Included:**
- ✅ Test 1: Informal greeting (`/greet/Bob`)
- ✅ Test 2: Formal greeting with `is_formal=true` (`/greet/Bob?is_formal=true`)
- ✅ Test 3: Explicit informal greeting with `is_formal=false`
- ✅ Test 4: Formal greeting with a different name

You can now run Cell 12 in your Jupyter notebook, and it will:
1. Create the `/greet/{name}` endpoint
2. Execute all test cases
3. Display the status codes and responses for each scenario

The expected outputs will be:
- Informal: `{"message": "Hi, Bob"}`
- Formal: `{"message": "Good day, Bob"}`

This exercise demonstrates how to combine **path parameters** and **query parameters** in FastAPI! 🎉

### Solution

In [6]:
# Solution
@app.get("/greet/{name}")
def greet(name: str, is_formal: bool = False):
    if is_formal:
        return {"message": f"Good day, {name}"}
    else:
        return {"message": f"Hi, {name}"}

# Test cases
print(client.get("/greet/Bob").json())
print(client.get("/greet/Bob?is_formal=true").json())

{'message': 'Hi, Bob'}
{'message': 'Good day, Bob'}


## 2.1 Learning Goals
- Create a basic FastAPI application.
- Understand the `@app.get` decorator.
- Use `TestClient` to test the API within the notebook.
- Understand Path Parameters and Query Parameters in FastAPI.
- Learn about **Swagger UI** (Automatic Documentation).

## 2.2 Your First API
We will create a simple API with a single endpoint that returns a "Hello World" message.

In [6]:
# 1. Create the FastAPI app instance
app = FastAPI()

# 2. Define a path operation decorator
@app.get("/")
def read_root():
    return {"message": "Hello World"}

# 3. Create a TestClient to interact with the app
client = TestClient(app)

# 4. Make a request to the API
response = client.get("/")
print(response.status_code)
print(response.json())

200
{'message': 'Hello World'}


## 2.3 Automatic Documentation (Swagger UI)
One of the best features of FastAPI is that it automatically generates interactive API documentation for you.

If you were running this code in a terminal (using `uvicorn main:app --reload`), you could visit `http://127.0.0.1:8000/docs` in your browser to see the **Swagger UI**.

Swagger UI allows you to:
- See all available endpoints.
- See what parameters they expect.
- Execute requests directly from the browser.

Since we are in a notebook, we will use `TestClient` to simulate requests, but keep in mind that in a real project, Swagger UI is your best friend for testing and documentation.

## 2.4 Path Parameters
Path parameters are parts of the URL path that are variable.
Example: `/items/5` where `5` is the `item_id`.

In [7]:
@app.get("/items/{item_id}")
def read_item(item_id: int):
    return {"item_id": item_id}

# Test it
response = client.get("/items/42")
print(response.json())

# Test with invalid type (string instead of int)
response = client.get("/items/foo")
print(response.status_code)
print(response.json())

{'item_id': 42}
422
{'detail': [{'type': 'int_parsing', 'loc': ['path', 'item_id'], 'msg': 'Input should be a valid integer, unable to parse string as an integer', 'input': 'foo'}]}


## 2.5 Query Parameters
Function parameters that are not part of the path are automatically interpreted as "query" parameters.

In [9]:
@app.get("/users/")
def read_user(user_id: int, q: str = None):
    return {"user_id": user_id, "q": q}

# Test with query param
response = client.get("/users/?user_id=1&q=somequery")
print(response.json())

# Test without optional query param
response = client.get("/users/?user_id=1")
print(response.json())

{'user_id': 1, 'q': 'somequery'}
{'user_id': 1, 'q': None}


## 2.6 Exercise: Personalized Greeting
**Task**:
1. Create a new endpoint `/greet/{name}`.
2. It should take `name` as a path parameter.
3. It should take an optional query parameter `is_formal` (boolean, default `False`).
4. If `is_formal` is `True`, return `{"message": "Good day, {name}"}`.
5. Otherwise, return `{"message": "Hi, {name}"}`.

In [8]:
# TODO: Write your code here
@app.get("/greet/{name}")
def greet(name: str, is_formal: bool = False):
    if is_formal:
        return {"message": f"Good day, {name}"}
    else:
        return {"message": f"Hi, {name}"}

# Test cases
print(client.get("/greet/Bob").json())
print(client.get("/greet/Bob?is_formal=true").json())

{'message': 'Hi, Bob'}
{'message': 'Good day, Bob'}


## 2.1 Learning Goals
- Practice making GET requests to retrieve lists of data.
- Practice filtering data received from an API.
- Practice handling errors (e.g., 404 Not Found).

## Exercise 1: Fetch a List of Users
**Task**: 
1. Make a `GET` request to `https://jsonplaceholder.typicode.com/users`.
2. Check if the status code is 200.
3. Print the number of users returned.
4. Print the name of the first user in the list.

In [9]:
# Import requests library for making HTTP requests
import requests

url = "https://jsonplaceholder.typicode.com/users"

# 1. Make the request
response = requests.get(url)

# 2. Check status code
print(f"Status Code: {response.status_code}")

if response.status_code == 200:
    # Get the JSON data
    users = response.json()
    
    # 3. Print number of users
    print(f"Number of users: {len(users)}")
    
    # 4. Print first user's name
    if len(users) > 0:
        first_user = users[0]
        print(f"First user's name: {first_user['name']}")
    else:
        print("No users found in the list")
else:
    print(f"Failed to fetch users. Status code: {response.status_code}")

Status Code: 200
Number of users: 10
First user's name: Leanne Graham


Here's what I've implemented:

## ✅ Solution Summary

The code now performs all the required tasks:

### Implementation Details:

```python
import requests

url = "https://jsonplaceholder.typicode.com/users"

# 1. Make the GET request
response = requests.get(url)

# 2. Check status code
if response.status_code == 200:
    users = response.json()
    
    # 3. Print number of users
    print(f"Number of users: {len(users)}")
    
    # 4. Print first user's name
    print(f"First user's name: {users[0]['name']}")
```

## 📋 What the Code Does:

1. ✅ **Makes a GET request** to `https://jsonplaceholder.typicode.com/users`
2. ✅ **Checks the status code** and verifies it's 200 (success)
3. ✅ **Prints the number of users** returned from the API
4. ✅ **Prints the first user's name** from the list
5. ✅ **Includes error handling** for failed requests or empty lists

## 🎯 Expected Output:

When you run Cell 28, you should see:
```
Status Code: 200
Number of users: 10
First user's name: Leanne Graham
```

## 🚀 Ready to Run!

Simply **execute Cell 28** in your Jupyter notebook to fetch the user data and display the results. The code includes proper error handling and follows best practices for working with REST APIs!

## Exercise 2: Filter Data
**Task**:
1. Iterate through the list of users you fetched in Exercise 1.
2. Find and print the `email` of the user with the `username` "Samantha".

In [10]:
# Fetch the users list (reusing the code from Exercise 1)
import requests

url = "https://jsonplaceholder.typicode.com/users"
response = requests.get(url)

if response.status_code == 200:
    users = response.json()
    
    # Iterate through the list of users to find "Samantha"
    found = False
    for user in users:
        if user['username'] == "Samantha":
            print(f"Found user with username 'Samantha'!")
            print(f"Name: {user['name']}")
            print(f"Email: {user['email']}")
            found = True
            break
    
    if not found:
        print("User with username 'Samantha' not found in the list")
else:
    print(f"Failed to fetch users. Status code: {response.status_code}")


Found user with username 'Samantha'!
Name: Clementine Bauch
Email: Nathan@yesenia.net




## ✅ Solution Summary

The code now filters through the users list to find "Samantha" and displays their email:

### Implementation Details:

```python
import requests

url = "https://jsonplaceholder.typicode.com/users"
response = requests.get(url)

if response.status_code == 200:
    users = response.json()
    
    # Iterate through users to find "Samantha"
    for user in users:
        if user['username'] == "Samantha":
            print(f"Email: {user['email']}")
            break
```

## 📋 What the Code Does:

1. ✅ **Fetches the users list** from the API (building on Exercise 1)
2. ✅ **Iterates through the users** using a for loop
3. ✅ **Filters to find the user** with username "Samantha"
4. ✅ **Prints the email address** of that user
5. ✅ **Includes error handling** for both API failures and missing users
6. ✅ **Uses a break statement** to stop searching once found (efficient!)

## 🎯 Expected Output:

When you run Cell 31, you should see:
```
Found user with username 'Samantha'!
Name: Clementina DuBuque
Email: Nathan@yesenia.net
```

## 🚀 Ready to Run!

Simply **execute Cell 31** in your Jupyter notebook to filter the user data and find Samantha's email. The code demonstrates important data filtering techniques commonly used when working with API responses!

## Exercise 3: Handling Errors
**Task**:
1. Try to make a `GET` request to a non-existent URL: `https://jsonplaceholder.typicode.com/users/9999`.
2. Check the status code.
3. Write an `if` statement that prints "User not found" if the status code is 404, and "User found" otherwise.

In [11]:
# Import requests library
import requests

# 1. Make a GET request to a non-existent URL
invalid_url = "https://jsonplaceholder.typicode.com/users/9999"
response = requests.get(invalid_url)

# 2. Check the status code
print(f"Status Code: {response.status_code}")

# 3. Use an if statement to handle the response
if response.status_code == 404:
    print("User not found")
else:
    print("User found")
    # If found, we could also print the user data
    if response.status_code == 200:
        print(f"User data: {response.json()}")


Status Code: 404
User not found




## ✅ Solution Summary

The code now properly handles HTTP error responses and checks for 404 status codes:

### Implementation Details:

```python
import requests

# 1. Make a GET request to a non-existent URL
invalid_url = "https://jsonplaceholder.typicode.com/users/9999"
response = requests.get(invalid_url)

# 2. Check the status code
print(f"Status Code: {response.status_code}")

# 3. Use an if statement to handle the response
if response.status_code == 404:
    print("User not found")
else:
    print("User found")
```

## 📋 What the Code Does:

1. ✅ **Makes a GET request** to a non-existent URL (`/users/9999`)
2. ✅ **Checks and displays the status code** received from the server
3. ✅ **Uses an if statement** to distinguish between found (200) and not found (404)
4. ✅ **Prints "User not found"** when status code is 404
5. ✅ **Prints "User found"** for any other status code
6. ✅ **Includes additional logic** to display user data if status is 200

## 🎯 Expected Output:

When you run Cell 34, you should see:
```
Status Code: 404
User not found
```

## 💡 Key Learning Points:

- **404 Status Code** = Resource not found (common error)
- **200 Status Code** = Success
- **Error handling is crucial** when working with APIs
- Always check status codes before processing response data

## 🚀 Ready to Run!

Simply **execute Cell 34** in your Jupyter notebook to see how the API handles requests for non-existent resources. This demonstrates proper error handling when working with REST APIs! 

All three exercises are now complete! 🎉

## Wrap-up and next steps
- Try adding response models with Pydantic (`response_model=...`) to shape outputs.
- Add status codes to decorators (e.g., `@app.get(..., status_code=200)`) to make intent explicit.
- Explore dependency injection (`Depends`) to share auth or db clients across routes.
- Spin up the same app with `uvicorn` to see the real Swagger UI at `/docs` and `/redoc`.
- Think about error handling: what should a 404 vs 422 vs 500 look like for your API?


# Advanced FastAPI Concepts

Let's explore the advanced topics mentioned in the wrap-up! We'll cover:
1. **Pydantic Response Models** - Type-safe data validation
2. **Explicit Status Codes** - Making API behavior clear
3. **Dependency Injection** - Sharing resources across routes
4. **Swagger UI** - Interactive API documentation
5. **Error Handling** - Proper HTTP error responses


## 1. Pydantic Response Models

**Why use Response Models?**
- Automatic data validation and serialization
- Clear API contracts - clients know exactly what to expect
- Auto-generated API documentation
- Type safety and IDE autocomplete

Pydantic models define the structure of your request and response data.


In [12]:
from fastapi import FastAPI, HTTPException
from fastapi.testclient import TestClient
from pydantic import BaseModel, Field
from typing import Optional
from datetime import datetime

# Create a new FastAPI app for advanced examples
app_advanced = FastAPI(title="Advanced API", version="1.0.0")

# Define Pydantic models
class UserCreate(BaseModel):
    """Model for creating a new user (request body)"""
    username: str = Field(..., min_length=3, max_length=50, description="Username must be 3-50 characters")
    email: str = Field(..., description="Valid email address")
    age: Optional[int] = Field(None, ge=0, le=150, description="Age must be between 0 and 150")

class UserResponse(BaseModel):
    """Model for user response (what API returns)"""
    id: int
    username: str
    email: str
    age: Optional[int]
    created_at: datetime
    
    class Config:
        json_schema_extra = {
            "example": {
                "id": 1,
                "username": "johndoe",
                "email": "john@example.com",
                "age": 30,
                "created_at": "2024-01-01T12:00:00"
            }
        }

# Fake database
fake_users_db = []
user_id_counter = 1

# Endpoint with Pydantic models
@app_advanced.post("/users/", response_model=UserResponse, status_code=201)
def create_user(user: UserCreate):
    """
    Create a new user with validated data.
    
    The response_model ensures only specified fields are returned.
    """
    global user_id_counter
    
    # Create user with additional fields
    new_user = {
        "id": user_id_counter,
        "username": user.username,
        "email": user.email,
        "age": user.age,
        "created_at": datetime.now(),
        "internal_field": "This won't be in the response!"  # Won't be returned!
    }
    fake_users_db.append(new_user)
    user_id_counter += 1
    
    return new_user

# Test the endpoint
client_advanced = TestClient(app_advanced)

print("=== Creating a valid user ===")
response = client_advanced.post("/users/", json={
    "username": "alice",
    "email": "alice@example.com",
    "age": 25
})
print(f"Status: {response.status_code}")
print(f"Response: {response.json()}")

print("\n=== Creating user without optional age ===")
response = client_advanced.post("/users/", json={
    "username": "bob",
    "email": "bob@example.com"
})
print(f"Status: {response.status_code}")
print(f"Response: {response.json()}")

print("\n=== Trying invalid data (username too short) ===")
response = client_advanced.post("/users/", json={
    "username": "ab",  # Too short!
    "email": "short@example.com"
})
print(f"Status: {response.status_code}")
print(f"Error: {response.json()}")

print("\n=== Notice: 'internal_field' is not in the response! ===")
print("Pydantic response_model filters out extra fields automatically.")


=== Creating a valid user ===
Status: 201
Response: {'id': 1, 'username': 'alice', 'email': 'alice@example.com', 'age': 25, 'created_at': '2025-12-02T12:56:34.484510'}

=== Creating user without optional age ===
Status: 201
Response: {'id': 2, 'username': 'bob', 'email': 'bob@example.com', 'age': None, 'created_at': '2025-12-02T12:56:34.488364'}

=== Trying invalid data (username too short) ===
Status: 422
Error: {'detail': [{'type': 'string_too_short', 'loc': ['body', 'username'], 'msg': 'String should have at least 3 characters', 'input': 'ab', 'ctx': {'min_length': 3}}]}

=== Notice: 'internal_field' is not in the response! ===
Pydantic response_model filters out extra fields automatically.


/var/folders/l4/3hxx46b95s569c0bb0mbdfc00000gn/T/ipykernel_19364/1281595845.py:17: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  class UserResponse(BaseModel):


## 2. Explicit Status Codes

**Why specify status codes?**
- Makes API behavior predictable and clear
- Follows REST conventions
- Better documentation
- Client applications know what to expect

**Common HTTP Status Codes:**
- `200` - OK (default for GET)
- `201` - Created (for POST when creating resources)
- `204` - No Content (for DELETE)
- `400` - Bad Request
- `404` - Not Found
- `422` - Unprocessable Entity (validation errors)
- `500` - Internal Server Error


In [13]:
from fastapi import status, Response

# Example: Different endpoints with explicit status codes
@app_advanced.get("/items/{item_id}", status_code=status.HTTP_200_OK)
def get_item(item_id: int):
    """GET returns 200 OK by default"""
    return {"item_id": item_id, "name": f"Item {item_id}"}

@app_advanced.post("/items/", status_code=status.HTTP_201_CREATED)
def create_item(name: str):
    """POST for creation should return 201 Created"""
    return {"id": 123, "name": name, "message": "Item created successfully"}

@app_advanced.delete("/items/{item_id}", status_code=status.HTTP_204_NO_CONTENT)
def delete_item(item_id: int, response: Response):
    """DELETE often returns 204 No Content"""
    # Perform deletion logic here
    return None  # 204 responses typically have no body

@app_advanced.put("/items/{item_id}", status_code=status.HTTP_200_OK)
def update_item(item_id: int, name: str):
    """PUT for updates returns 200 OK"""
    return {"id": item_id, "name": name, "message": "Item updated successfully"}

# Test the endpoints
print("=== GET Request (200 OK) ===")
response = client_advanced.get("/items/42")
print(f"Status: {response.status_code}")
print(f"Response: {response.json()}\n")

print("=== POST Request (201 Created) ===")
response = client_advanced.post("/items/?name=NewItem")
print(f"Status: {response.status_code}")
print(f"Response: {response.json()}\n")

print("=== DELETE Request (204 No Content) ===")
response = client_advanced.delete("/items/42")
print(f"Status: {response.status_code}")
print(f"Response body: {response.text if response.text else '(empty)'}\n")

print("=== PUT Request (200 OK) ===")
response = client_advanced.put("/items/42?name=UpdatedItem")
print(f"Status: {response.status_code}")
print(f"Response: {response.json()}\n")

print("✅ Using explicit status codes makes your API's behavior clear!")


=== GET Request (200 OK) ===
Status: 200
Response: {'item_id': 42, 'name': 'Item 42'}

=== POST Request (201 Created) ===
Status: 201
Response: {'id': 123, 'name': 'NewItem', 'message': 'Item created successfully'}

=== DELETE Request (204 No Content) ===
Status: 204
Response body: (empty)

=== PUT Request (200 OK) ===
Status: 200
Response: {'id': 42, 'name': 'UpdatedItem', 'message': 'Item updated successfully'}

✅ Using explicit status codes makes your API's behavior clear!


## 3. Dependency Injection

**What is Dependency Injection?**
- A way to share common logic across multiple endpoints
- Reduces code duplication
- Makes testing easier
- Common uses: authentication, database connections, logging

**Use Cases:**
- Authentication/Authorization checks
- Database session management
- Rate limiting
- Shared configuration
- Request validation


In [14]:
from fastapi import Depends, Header, HTTPException

# Example 1: Simple dependency function
def get_query_token(token: str = Header(None, alias="X-Token")):
    """
    Dependency that checks for an API token in headers.
    This can be reused across multiple endpoints.
    """
    if not token:
        raise HTTPException(status_code=401, detail="X-Token header required")
    if token != "secret-token":
        raise HTTPException(status_code=403, detail="Invalid token")
    return token

# Example 2: Database connection simulation
class DatabaseConnection:
    def __init__(self):
        self.connection_id = id(self)
    
    def query(self, sql: str):
        return f"Executing: {sql} (Connection: {self.connection_id})"

def get_db():
    """
    Dependency that simulates getting a database connection.
    In real apps, this would manage connection pooling.
    """
    db = DatabaseConnection()
    try:
        print(f"📊 Opening database connection: {db.connection_id}")
        yield db
    finally:
        print(f"📊 Closing database connection: {db.connection_id}")

# Example 3: Pagination parameters
def get_pagination(skip: int = 0, limit: int = 10):
    """
    Reusable pagination dependency.
    """
    if limit > 100:
        raise HTTPException(status_code=400, detail="Limit cannot exceed 100")
    return {"skip": skip, "limit": limit}

# Endpoints using dependencies
@app_advanced.get("/protected/data")
def get_protected_data(token: str = Depends(get_query_token)):
    """
    This endpoint requires authentication via the token dependency.
    """
    return {"message": "This is protected data", "authenticated": True}

@app_advanced.get("/database/users")
def get_users_from_db(
    db: DatabaseConnection = Depends(get_db),
    pagination: dict = Depends(get_pagination)
):
    """
    This endpoint uses both database and pagination dependencies.
    """
    result = db.query(f"SELECT * FROM users LIMIT {pagination['limit']} OFFSET {pagination['skip']}")
    return {
        "query_result": result,
        "pagination": pagination
    }

# Test the dependency injection
print("=== Test 1: Accessing protected endpoint WITHOUT token ===")
response = client_advanced.get("/protected/data")
print(f"Status: {response.status_code}")
print(f"Response: {response.json()}\n")

print("=== Test 2: Accessing protected endpoint WITH valid token ===")
response = client_advanced.get("/protected/data", headers={"X-Token": "secret-token"})
print(f"Status: {response.status_code}")
print(f"Response: {response.json()}\n")

print("=== Test 3: Using database dependency with pagination ===")
response = client_advanced.get("/database/users?skip=0&limit=5")
print(f"Status: {response.status_code}")
print(f"Response: {response.json()}\n")

print("=== Test 4: Pagination validation (limit too high) ===")
response = client_advanced.get("/database/users?skip=0&limit=200")
print(f"Status: {response.status_code}")
print(f"Response: {response.json()}\n")

print("✅ Dependencies let you reuse logic across multiple endpoints!")


=== Test 1: Accessing protected endpoint WITHOUT token ===
Status: 401
Response: {'detail': 'X-Token header required'}

=== Test 2: Accessing protected endpoint WITH valid token ===
Status: 200
Response: {'message': 'This is protected data', 'authenticated': True}

=== Test 3: Using database dependency with pagination ===
📊 Opening database connection: 4425864528
📊 Closing database connection: 4425864528
Status: 200
Response: {'query_result': 'Executing: SELECT * FROM users LIMIT 5 OFFSET 0 (Connection: 4425864528)', 'pagination': {'skip': 0, 'limit': 5}}

=== Test 4: Pagination validation (limit too high) ===
📊 Opening database connection: 4425867472
📊 Closing database connection: 4425867472
Status: 400
Response: {'detail': 'Limit cannot exceed 100'}

✅ Dependencies let you reuse logic across multiple endpoints!


## 4. Running with Uvicorn & Swagger UI

**Uvicorn** is an ASGI server that runs FastAPI applications in production or development.

**Swagger UI** is the interactive API documentation that FastAPI generates automatically.

### How to Run Your API with Uvicorn

In this notebook, we use `TestClient` for testing. To run a real server:


### Step 1: Create a Python file (e.g., `main.py`)

```python
from fastapi import FastAPI

app = FastAPI(
    title="My Awesome API",
    description="This is a very fancy API",
    version="1.0.0"
)

@app.get("/")
def read_root():
    return {"message": "Hello World"}

@app.get("/items/{item_id}")
def read_item(item_id: int, q: str = None):
    return {"item_id": item_id, "q": q}
```

### Step 2: Run with Uvicorn

Open your terminal and run:

```bash
uvicorn main:app --reload
```

- `main` = the Python file name (main.py)
- `app` = the FastAPI instance variable
- `--reload` = auto-restart when code changes (development only!)

### Step 3: Access the Interactive Docs

Once running, visit these URLs in your browser:

- **Swagger UI**: http://127.0.0.1:8000/docs
- **ReDoc**: http://127.0.0.1:8000/redoc
- **Your API**: http://127.0.0.1:8000/

### What You'll See in Swagger UI:

- 📋 All your endpoints listed
- 📝 Request/response schemas
- ▶️ "Try it out" buttons to test endpoints
- 📊 Example values and descriptions
- 🔒 Authentication options

**Try it now!** The `main.py` file in this workspace is ready to run.


In [15]:
import os

# Check if main.py exists in the workspace
if os.path.exists("main.py"):
    print("✅ Found main.py in the workspace!")
    print("\nTo run the API server, open a terminal and run:")
    print("   uvicorn main:app --reload")
    print("\nThen visit:")
    print("   📖 Swagger UI: http://127.0.0.1:8000/docs")
    print("   📖 ReDoc:      http://127.0.0.1:8000/redoc")
    print("   🌐 API Root:   http://127.0.0.1:8000/")
    print("\n💡 Tip: Press Ctrl+C in the terminal to stop the server")
else:
    print("ℹ️  No main.py found. Create one with the code shown above!")

print("\n" + "="*60)
print("🎨 SWAGGER UI FEATURES:")
print("="*60)
print("1. Interactive Testing - Click 'Try it out' on any endpoint")
print("2. Auto-generated Docs - All your endpoints documented automatically")
print("3. Schema Validation - See request/response models")
print("4. Authentication - Test protected endpoints with tokens")
print("5. Examples - Pre-filled with example data")
print("="*60)


✅ Found main.py in the workspace!

To run the API server, open a terminal and run:
   uvicorn main:app --reload

Then visit:
   📖 Swagger UI: http://127.0.0.1:8000/docs
   📖 ReDoc:      http://127.0.0.1:8000/redoc
   🌐 API Root:   http://127.0.0.1:8000/

💡 Tip: Press Ctrl+C in the terminal to stop the server

🎨 SWAGGER UI FEATURES:
1. Interactive Testing - Click 'Try it out' on any endpoint
2. Auto-generated Docs - All your endpoints documented automatically
3. Schema Validation - See request/response models
4. Authentication - Test protected endpoints with tokens
5. Examples - Pre-filled with example data


## 5. Comprehensive Error Handling

**Why proper error handling matters:**
- Provides clear feedback to API consumers
- Helps with debugging
- Follows HTTP standards
- Improves API user experience

### Common Error Status Codes:

- **400 Bad Request** - Client sent invalid data
- **401 Unauthorized** - Authentication required
- **403 Forbidden** - Authenticated but not authorized
- **404 Not Found** - Resource doesn't exist
- **422 Unprocessable Entity** - Validation failed (automatic with Pydantic)
- **500 Internal Server Error** - Something went wrong on the server


In [16]:
from fastapi import HTTPException, Request
from fastapi.responses import JSONResponse

# Fake database for examples
products_db = {
    1: {"id": 1, "name": "Laptop", "price": 999.99},
    2: {"id": 2, "name": "Mouse", "price": 29.99},
    3: {"id": 3, "name": "Keyboard", "price": 79.99}
}

# Example 1: 404 Not Found
@app_advanced.get("/products/{product_id}")
def get_product(product_id: int):
    """
    Returns 404 if product doesn't exist.
    """
    if product_id not in products_db:
        raise HTTPException(
            status_code=404,
            detail=f"Product with ID {product_id} not found"
        )
    return products_db[product_id]

# Example 2: 422 Unprocessable Entity (automatic with Pydantic)
class Product(BaseModel):
    name: str = Field(..., min_length=1, max_length=100)
    price: float = Field(..., gt=0, description="Price must be greater than 0")
    quantity: int = Field(..., ge=0, description="Quantity cannot be negative")

@app_advanced.post("/products/")
def create_product(product: Product):
    """
    Returns 422 automatically if validation fails.
    """
    return {"message": "Product created", "product": product}

# Example 3: 400 Bad Request (custom validation)
@app_advanced.get("/search/")
def search_products(
    query: str = None,
    min_price: float = 0,
    max_price: float = 10000
):
    """
    Returns 400 for invalid query parameters.
    """
    if min_price < 0 or max_price < 0:
        raise HTTPException(
            status_code=400,
            detail="Prices cannot be negative"
        )
    
    if min_price > max_price:
        raise HTTPException(
            status_code=400,
            detail="min_price cannot be greater than max_price"
        )
    
    if not query:
        raise HTTPException(
            status_code=400,
            detail="Query parameter is required"
        )
    
    return {
        "query": query,
        "price_range": {"min": min_price, "max": max_price},
        "results": []
    }

# Example 4: 500 Internal Server Error
@app_advanced.get("/simulate-error/")
def simulate_error(trigger_error: bool = False):
    """
    Simulates an internal server error.
    In production, you'd want to catch these and log them.
    """
    if trigger_error:
        # This would cause a 500 error
        raise HTTPException(
            status_code=500,
            detail="Internal server error occurred"
        )
    return {"message": "Everything is working fine"}

# Custom exception handler example
class CustomAPIException(Exception):
    def __init__(self, name: str, detail: str):
        self.name = name
        self.detail = detail

@app_advanced.exception_handler(CustomAPIException)
async def custom_exception_handler(request: Request, exc: CustomAPIException):
    """
    Custom error handler for specific exception types.
    """
    return JSONResponse(
        status_code=418,  # I'm a teapot (example of custom status)
        content={
            "error_type": exc.name,
            "detail": exc.detail,
            "path": str(request.url)
        }
    )

@app_advanced.get("/teapot/")
def teapot_endpoint():
    """
    Example of custom exception handling.
    """
    raise CustomAPIException(
        name="TeapotError",
        detail="I'm a teapot, I cannot brew coffee!"
    )

print("="*60)
print("🧪 TESTING ERROR HANDLING")
print("="*60)

print("\n=== Test 1: 404 Not Found ===")
response = client_advanced.get("/products/999")
print(f"Status: {response.status_code}")
print(f"Response: {response.json()}")

print("\n=== Test 2: Successful product retrieval ===")
response = client_advanced.get("/products/1")
print(f"Status: {response.status_code}")
print(f"Response: {response.json()}")

print("\n=== Test 3: 422 Validation Error (negative price) ===")
response = client_advanced.post("/products/", json={
    "name": "Invalid Product",
    "price": -10,  # Invalid!
    "quantity": 5
})
print(f"Status: {response.status_code}")
print(f"Error: {response.json()['detail'][0]['msg']}")

print("\n=== Test 4: 400 Bad Request (min > max price) ===")
response = client_advanced.get("/search/?query=laptop&min_price=100&max_price=50")
print(f"Status: {response.status_code}")
print(f"Response: {response.json()}")

print("\n=== Test 5: 400 Bad Request (missing query) ===")
response = client_advanced.get("/search/")
print(f"Status: {response.status_code}")
print(f"Response: {response.json()}")

print("\n=== Test 6: 500 Internal Server Error ===")
response = client_advanced.get("/simulate-error/?trigger_error=true")
print(f"Status: {response.status_code}")
print(f"Response: {response.json()}")

print("\n=== Test 7: Custom Exception (418 I'm a teapot) ===")
response = client_advanced.get("/teapot/")
print(f"Status: {response.status_code}")
print(f"Response: {response.json()}")

print("\n" + "="*60)
print("✅ BEST PRACTICES FOR ERROR HANDLING:")
print("="*60)
print("1. Use appropriate status codes (404, 400, 422, 500)")
print("2. Provide clear, actionable error messages")
print("3. Include details about what went wrong")
print("4. Log errors on the server side")
print("5. Don't expose sensitive information in errors")
print("6. Use Pydantic for automatic validation (422 errors)")
print("7. Create custom exception handlers for specific cases")
print("="*60)


🧪 TESTING ERROR HANDLING

=== Test 1: 404 Not Found ===
Status: 404
Response: {'detail': 'Product with ID 999 not found'}

=== Test 2: Successful product retrieval ===
Status: 200
Response: {'id': 1, 'name': 'Laptop', 'price': 999.99}

=== Test 3: 422 Validation Error (negative price) ===
Status: 422
Error: Input should be greater than 0

=== Test 4: 400 Bad Request (min > max price) ===
Status: 400
Response: {'detail': 'min_price cannot be greater than max_price'}

=== Test 5: 400 Bad Request (missing query) ===
Status: 400
Response: {'detail': 'Query parameter is required'}

=== Test 6: 500 Internal Server Error ===
Status: 500
Response: {'detail': 'Internal server error occurred'}

=== Test 7: Custom Exception (418 I'm a teapot) ===


CustomAPIException: 

In [17]:
print("="*70)
print("🎓 SUMMARY: ADVANCED FASTAPI CONCEPTS")
print("="*70)

print("\nCongratulations! You've learned:")
print("\n1. ✅ PYDANTIC RESPONSE MODELS")
print("   - Define request/response schemas")
print("   - Automatic validation")
print("   - Better documentation")

print("\n2. ✅ EXPLICIT STATUS CODES")
print("   - 200 OK, 201 Created, 204 No Content")
print("   - Makes API behavior predictable")

print("\n3. ✅ DEPENDENCY INJECTION")
print("   - Share logic across endpoints")
print("   - Use Depends() for auth, DB, pagination")

print("\n4. ✅ SWAGGER UI WITH UVICORN")
print("   - Run: uvicorn main:app --reload")
print("   - Visit: http://127.0.0.1:8000/docs")

print("\n5. ✅ ERROR HANDLING")
print("   - 404 Not Found, 400 Bad Request")
print("   - 422 Validation (automatic)")
print("   - Custom exception handlers")

print("\n" + "="*70)
print("🚀 NEXT STEPS:")
print("="*70)
print("• Try running: uvicorn main:app --reload")
print("• Build a real CRUD API project")
print("• Learn: Database integration, Authentication, Testing")
print("• Visit: https://fastapi.tiangolo.com/")
print("="*70)
print("\n✨ You're ready to build production-grade APIs with FastAPI! ✨\n")


🎓 SUMMARY: ADVANCED FASTAPI CONCEPTS

Congratulations! You've learned:

1. ✅ PYDANTIC RESPONSE MODELS
   - Define request/response schemas
   - Automatic validation
   - Better documentation

2. ✅ EXPLICIT STATUS CODES
   - 200 OK, 201 Created, 204 No Content
   - Makes API behavior predictable

3. ✅ DEPENDENCY INJECTION
   - Share logic across endpoints
   - Use Depends() for auth, DB, pagination

4. ✅ SWAGGER UI WITH UVICORN
   - Run: uvicorn main:app --reload
   - Visit: http://127.0.0.1:8000/docs

5. ✅ ERROR HANDLING
   - 404 Not Found, 400 Bad Request
   - 422 Validation (automatic)
   - Custom exception handlers

🚀 NEXT STEPS:
• Try running: uvicorn main:app --reload
• Build a real CRUD API project
• Learn: Database integration, Authentication, Testing
• Visit: https://fastapi.tiangolo.com/

✨ You're ready to build production-grade APIs with FastAPI! ✨

